<a href="https://colab.research.google.com/github/yeonshiri/AGS/blob/main/code/short_answer_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --------------------------------------------------------------
# PaddleOcr

!pip install paddlepaddle
!pip install paddleocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: opt_einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=9179ad69378fdc18dc50c02068fd702eafb25065a63db406853506e63018852c
  Stored 

In [ ]:
import os
import cv2
from paddleocr import PaddleOCR
import matplotlib.pyplot as plt

# 1. 경로 설정
image_path = "/content/yolov5/images/valid/image_shortanswer_73_roi1_class1.jpg"
label_path = "/content/yolov5/labels/valid/image_shortanswer_73_roi1_class1.txt"

# 2. 이미지 읽기
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"이미지 로드 실패: {image_path}")
h, w = image.shape[:2]

# 3. OCR 초기화
ocr = PaddleOCR(use_angle_cls=False, lang='en')

# 4. 라벨 파일 읽고 ROI 추출 + OCR 수행
with open(label_path, 'r') as f:
    lines = f.readlines()

for idx, line in enumerate(lines):
    parts = line.strip().split()
    if len(parts) < 5:
        print(f"라벨 오류: {line.strip()}")
        continue

    # YOLO 좌표 → 픽셀 좌표
    _, x_center, y_center, box_w, box_h = map(float, parts)
    x1 = int((x_center - box_w / 2) * w)
    y1 = int((y_center - box_h / 2) * h)
    x2 = int((x_center + box_w / 2) * w)
    y2 = int((y_center + box_h / 2) * h)

    roi = image[y1:y2, x1:x2]
    if roi.size == 0:
        print(f"빈 ROI 스킵: {x1,y1,x2,y2}")
        continue

    # OCR 수행
    result = ocr.ocr(roi, cls=False)

    # 결과 출력
    print(f"\n ROI #{idx} 위치: {x1, y1, x2, y2}")
    for line in result:
        for box in line:
            text = box[1][0]
            conf = box[1][1]
            print(f"   인식: '{text}' (정확도: {conf:.2f})")

    # 시각화 (선택)
    plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
    plt.title(f"ROI #{idx}: '{text}'")
    plt.axis('off')
    plt.show()


[2025/05/18 02:39:43] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_l